In [129]:
from draco.learn import linear
from draco.learn import data_util
from draco.run import run
from draco.spec import Task, Query
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [130]:
test_size = 0.3

In [131]:
data, _ = data_util.load_data(test_size=test_size)
data = data[data.source == 'younghoon']

raw_data = data_util.load_neg_pos_specs()

In [132]:
X = data.positive - data.negative
y = np.ones(len(X))

In [133]:
clf = linear.train_model(X, test_size=test_size)
pred = clf.predict(X)

Train score:  0.9273049645390071
Dev score:  0.9259259259259259


In [134]:
weights = clf.coef_[0]
names = X.columns

In [181]:
const_dict = dict(zip(map(lambda x: f'{x}_weight', names), map(int, weights*1000)))
weight_dict = dict(zip(names, weights))

const_dict, weight_dict

({'agg_dim_weight': 0,
  'aggregate_count_weight': 0,
  'aggregate_group_by_raw_weight': 0,
  'aggregate_max_weight': 0,
  'aggregate_mean_weight': 0,
  'aggregate_median_weight': 0,
  'aggregate_min_weight': 0,
  'aggregate_stdev_weight': 0,
  'aggregate_weight': 0,
  'bin_cardinality_weight': 0,
  'bin_high_weight': 0,
  'bin_low_weight': 0,
  'bin_weight': 0,
  'c_c_area_weight': 0,
  'c_c_line_weight': 0,
  'c_c_point_weight': 18,
  'c_c_text_weight': 0,
  'c_c_tick_weight': 0,
  'c_d_area_weight': 0,
  'c_d_bar_weight': 0,
  'c_d_column_weight': 0,
  'c_d_line_weight': 0,
  'c_d_no_overlap_area_weight': 0,
  'c_d_no_overlap_bar_weight': 0,
  'c_d_no_overlap_line_weight': 0,
  'c_d_no_overlap_point_weight': 0,
  'c_d_no_overlap_text_weight': 0,
  'c_d_no_overlap_tick_weight': 0,
  'c_d_point_weight': -18,
  'c_d_text_weight': 0,
  'c_d_tick_weight': 0,
  'categorical_cardinality_weight': 0,
  'continuous_color_weight': 287,
  'continuous_size_weight': -306,
  'continuous_text_weigh

In [177]:
only_correct = X[pred == 1]

len(only_correct), len(data)

(748, 807)

In [193]:
files = ['define.lp', 'test.lp', 'features.lp', 'count.lp', 'output.lp']

def aggregate(violations):
    ''' Get the cost from a dictionary of violations. '''
    ret = 0.0
    for n, v in violations.items():
        ret += weight_dict[n] * v
    return ret

for i in range(len(only_correct))[:10]:
    name = only_correct.iloc[i].name
    
    print(name)
    
    pair = raw_data[name]

    task_n = Task(pair.data, Query.from_vegalite(pair.negative))
    task_p = Task(pair.data, Query.from_vegalite(pair.positive))
    
    assert clf.predict([entry]) == 1
    
    data_entry = data.loc[name]
    # print("Prediction negative", clf.coef_.dot(np.array([data_entry.negative]).T))
    print("Prediction negative:", clf.decision_function([data_entry.negative])[0])
    # print("Prediction positive", clf.coef_.dot(np.array([data_entry.positive]).T))
    print("Prediction positive:", clf.decision_function([data_entry.positive])[0])
    
#     cn = run(task_n, constants=const_dict).cost
#     cp = run(task_p, constants=const_dict).cost
#     print(cn, cp, cn > cp)
    
    cn = aggregate(run(task_n, files=files, silence_warnings=True).violations)
    cp = aggregate(run(task_p, files=files, silence_warnings=True).violations)
    
    print(cn, cp, cn > cp)
    
    print

younghoon-664
Prediction negative: -0.4764404722559446
Prediction positive: 0.9723757503098334
-0.0014407701675249879 0.6769377813650127 False
younghoon-955
Prediction negative: -1.166583756002711
Prediction positive: 0.8674073599768667
-0.472443278575694 0.571969391032046 False
younghoon-1016
Prediction negative: -1.166583756002711
Prediction positive: 1.1135468529618713
-0.472443278575694 0.2398446423912579 False
younghoon-843
Prediction negative: -1.166583756002711
Prediction positive: 1.1135468529618713
-0.472443278575694 0.2398446423912579 False
younghoon-327
Prediction negative: 0.4443511986393648
Prediction positive: 0.9359506175936795
0.4470663108482781 0.31645835234652875 True
younghoon-1043
Prediction negative: -0.2155182210433184
Prediction positive: 1.104766184099248
-0.07262639522718228 1.0823050305300144 False
younghoon-991
Prediction negative: -0.467270750366897
Prediction positive: 1.104766184099248
-0.3468400781199943 1.0823050305300144 False
younghoon-689
Prediction n